<a href="https://colab.research.google.com/github/vsuhas9/StyleTransfer/blob/dev-suhas/Style%20Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [4]:
# Package to handle files
import os

# Image Handling
import numpy as np

# Memory Handling
import gc
from numba import cuda

# Tensorflow Componenets
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19
from tensorflow.keras.backend import clear_session

# Few Initial Configuration

In [5]:
result_prefix = "generated"

# Weights of the different loss components
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

# Dimensions of the generated picture.
img_nrows = 600
#img_ncols = int(width * img_nrows / height)

# Download the Transfer Images and Style Images

In [6]:
!rm -rf target_images.zip
!rm -rf target_images

!rm -rf style_images.zip
!rm -rf style_images

!rm -rf outputs

!wget https://raw.githubusercontent.com/vsuhas9/StyleTransfer/dev-suhas/Images/target/target_images.zip
!unzip /content/target_images.zip

!wget https://raw.githubusercontent.com/vsuhas9/StyleTransfer/dev-suhas/Images/style/style_images.zip
!unzip /content/style_images.zip

!mkdir outputs

--2024-01-18 18:18:46--  https://raw.githubusercontent.com/vsuhas9/StyleTransfer/dev-suhas/Images/target/target_images.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5468160 (5.2M) [application/zip]
Saving to: ‘target_images.zip’

target_images.zip   100%[===================>]   5.21M  --.-KB/s    in 0.03s   

2024-01-18 18:18:46 (178 MB/s) - ‘target_images.zip’ saved [5468160/5468160]

Archive:  /content/target_images.zip
   creating: target_images/
  inflating: target_images/Masterlayer_Event221_SetA.png  
--2024-01-18 18:18:46--  https://raw.githubusercontent.com/vsuhas9/StyleTransfer/dev-suhas/Images/style/style_images.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connec

# Image paths and other componenets

In [7]:
def list_files(folder_path):
    files = []
    for entry in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, entry)):
            files.append( folder_path + entry)
    return files

# Example usage
target_folder_path = '/content/target_images/'
style_folder_path = '/content/style_images/'

try:
  target_images =  list_files(target_folder_path)
  style_images = list_files(style_folder_path)

  print(str(len(target_images)) + " Target Images loaded & " + str(len(style_images)) + " Style Images loaded" )
except:
  print("Error loading the files")

1 Target Images loaded & 1 Style Images loaded


# Image Pre-Processing

In [8]:
def preprocess_image(image_path, img_nrows, img_ncols):
    # Util function to open, resize and format
    # pictures into appropriate tensors
    img = keras.preprocessing.image.load_img(
        image_path, target_size=(img_nrows, img_ncols)
    )
    img = keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)


def deprocess_image(x, img_nrows, img_ncols):
    # Util function to convert a tensor into a valid image
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype("uint8")
    return x

# Gram Matrix and Loss Calculation Functions

In [9]:
def gram_matrix(x, img_nrows, img_ncols):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style, combination, img_nrows, img_ncols):
    S = gram_matrix(style, img_nrows, img_ncols)
    C = gram_matrix(combination, img_nrows, img_ncols)
    channels = 3
    size = img_nrows * img_ncols
    return tf.reduce_sum(tf.square(S - C)) /\
      (4.0 * (channels ** 2) * (size ** 2))

def content_loss(base, combination):
    return tf.reduce_sum(tf.square(combination - base))

def total_variation_loss(x, img_nrows, img_ncols):
    a = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] \
          - x[:, 1:, : img_ncols - 1, :]
    )
    b = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] \
          - x[:, : img_nrows - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))

# VGG Neural Network

In [10]:
def compute_loss(combination_image, base_image, style_reference_image, img_nrows, img_ncols, feature_extractor, content_layer_name, style_layer_names):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)

    # Initialize the loss
    loss = tf.zeros(shape=())

    # Add content loss
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    # Add style loss
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features, img_nrows, img_ncols)
        loss += (style_weight / len(style_layer_names)) * sl

    # Add total variation loss
    loss += total_variation_weight * \
      total_variation_loss(combination_image, img_nrows, img_ncols)
    return loss

@tf.function
def compute_loss_and_grads(combination_image, \
                  base_image, style_reference_image, img_nrows, img_ncols, feature_extractor, content_layer_name, style_layer_names):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, \
                base_image, style_reference_image, img_nrows, img_ncols, feature_extractor, content_layer_name, style_layer_names)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

In [11]:
def get_styled_images(target_image_address, style_image_address,):

  width, height = keras.preprocessing.image.load_img(target_image_address).size
  img_nrows = 400
  img_ncols = int(width * img_nrows / height)

  target_name = target_image_address.replace(target_folder_path, "")
  style_image = style_image_address.replace(style_folder_path, "")

  base_image = preprocess_image(target_image_address, img_nrows, img_ncols)
  style_reference_image = preprocess_image(style_image_address, img_nrows, img_ncols)
  combination_image = tf.Variable(preprocess_image(target_image_address, img_nrows, img_ncols))
  print("Current Image is :" + target_name + " with the style: " + style_image)


  model = vgg19.VGG19(weights="imagenet", include_top=False)
  outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
  feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

  # List of layers to use for the style loss.
  style_layer_names = [
      "block1_conv1",
      "block2_conv1",
      "block3_conv1",
      "block4_conv1",
      "block5_conv1",
  ]
  # The layer to use for the content loss.
  content_layer_name = "block5_conv2"

  optimizer = keras.optimizers.SGD(
      keras.optimizers.schedules.ExponentialDecay(
          initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
      )
  )

  iterations = 100
  for i in range(1, iterations + 1):
      loss, grads = compute_loss_and_grads(
          combination_image, base_image, style_reference_image, img_nrows, img_ncols, feature_extractor, content_layer_name, style_layer_names
      )
      optimizer.apply_gradients([(grads, combination_image)])
      if i % 100 == 0:

          print("Iteration %d: loss=%.2f" % (i, loss))
          img = deprocess_image(combination_image.numpy(), img_nrows, img_ncols)
          fname = "/content/outputs/" + target_name + "_" + style_image + "_" + result_prefix + "_at_iteration_%d.png" % i
          keras.preprocessing.image.save_img(fname, img)
  clear_session()
  del model, optimizer, feature_extractor
  gc.collect()


In [12]:
for target_image_address in target_images:
  for style_image_address in style_images:
    get_styled_images(target_image_address, style_image_address)
   # device = cuda.get_current_device()
   # device.reset()

Current Image is :Masterlayer_Event221_SetA.png with the style: ghost_demon.jpg
Iteration 100: loss=6358.52


In [13]:
!/opt/bin/nvidia-smi

Thu Jan 18 18:19:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0              35W /  70W |   7421MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
!ps -aux|grep python

root          59  0.2  0.0      0     0 ?        Z    17:44   0:06 [python3] <defunct>
root          60  0.0  0.3  67772 52376 ?        S    17:44   0:00 python3 /usr/local/bin/colab-file
root         106  0.2  0.8 357808 113772 ?       Rl   17:44   0:05 /usr/bin/python3 /usr/local/bin/j
root        6252  5.0 11.9 18256952 1591376 ?    Ssl  18:07   0:39 /usr/bin/python3 -m colab_kernel_
root        6277  0.0  0.1 539276 14288 ?        Sl   18:07   0:00 /usr/bin/python3 /usr/local/lib/p
root        9579  0.0  0.0   7376  3452 ?        S    18:20   0:00 /bin/bash -c ps -aux|grep python
root        9581  0.0  0.0   6484  2300 ?        S    18:20   0:00 grep python
